In [14]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [15]:
%cd yolov5 (1)

[WinError 2] The system cannot find the file specified: 'yolov5 (1)'
d:\Downloads\yolov5\yolov5


In [16]:
%pip install -r "d:/Downloads/yolov5/yolov5"

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 13] Permission denied: 'd:/Downloads/yolov5/yolov5'


In [17]:
import sys
import platform
import pathlib
from pathlib import Path
import torch
import cv2
import numpy as np

# Fix untuk error PosixPath di Windows
if platform.system() == 'Windows':
    pathlib.PosixPath = pathlib.WindowsPath

# Path ke YOLOv5
YOLOV5_PATH = Path('yolov5s.pt').resolve()
if str(YOLOV5_PATH) not in sys.path:
    sys.path.append(str(YOLOV5_PATH))

# Import dari YOLOv5 lokal
from utils.augmentations import letterbox
from utils.general import non_max_suppression
from models.common import DetectMultiBackend

# Custom scale_coords karena versi baru tidak menyediakannya
def scale_coords(img1_shape, coords, img0_shape, ratio_pad=None):
    if ratio_pad is None:
        gain = min(img1_shape[0] / img0_shape[0], img1_shape[1] / img0_shape[1])
        pad = ((img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2)
    else:
        gain = ratio_pad[0][0]
        pad = ratio_pad[1]

    coords[:, [0, 2]] -= pad[0]
    coords[:, [1, 3]] -= pad[1]
    coords[:, :4] /= gain
    coords[:, 0].clamp_(0, img0_shape[1])
    coords[:, 1].clamp_(0, img0_shape[0])
    coords[:, 2].clamp_(0, img0_shape[1])
    coords[:, 3].clamp_(0, img0_shape[0])
    return coords

# Fungsi menggambar satu bounding box
def plot_one_box(xyxy, img, color=(0, 255, 0), label=None, line_thickness=2):
    tl = line_thickness or max(round(0.002 * (img.shape[0] + img.shape[1]) / 2), 1)
    c1, c2 = (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2_label = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2_label, color, -1, cv2.LINE_AA)
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, (255, 255, 255), thickness=tf, lineType=cv2.LINE_AA)

# Path model hasil training kamu
weights = r"runs/train/yolov5s_results2/weights/best.pt"

# Load model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DetectMultiBackend(weights, device=device)
model.eval()
names = model.names

# Mulai webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print(' Gagal menangkap frame dari webcam')
        break

    # Preprocess input
    img = letterbox(frame, new_shape=640)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR ke RGB dan HWC ke CHW
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device).float() / 255.0
    img = img.unsqueeze(0)

    # Inference
    with torch.no_grad():
        pred = model(img)
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)[0]

    # Gambar hasil
    if pred is not None and len(pred):
        pred[:, :4] = scale_coords(img.shape[2:], pred[:, :4], frame.shape).round()
        for *xyxy, conf, cls in pred:
            label = f'{names[int(cls)]} {conf:.2f}'
            plot_one_box(xyxy, frame, label=label)

    cv2.imshow('YOLOv5 Realtime Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print(" Exit requested.")
        break

cap.release()
cv2.destroyAllWindows()


Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7257306 parameters, 0 gradients


 Exit requested.
